# 02 Tax change analysis

Using data files output by code in `01-join-property-components.ipynb`.

In [1]:
# Libraries
import pandas as pd
import json

# Display options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_row', 300)
pd.options.display.float_format = '{:,.2f}'.format

# Custom aggregation functions
def per25 (g):
    return g.quantile(0.25)
def per50 (g):
    return g.quantile(0.50)
def per75 (g):
    return g.quantile(0.75)

In [2]:
# Data imports
dtype_joined = {
    'PropertyNumber': str,
    'Co': int,
    'ClassCodeDesc_23': str,
    'PropertyTypeDesc_23': str,
    'PropertyCat_mtfp_23': str, # MTFP-assigned in earlier step
    'Mills_23': str,
    'MV_23': float,
    'TV_23': float,
    'Est_Taxes_23': float,
    'ClassCodeDesc_22': str,
    'PropertyTypeDesc_22': str,
    'Mills_22': str,
    'MV_22': float,
    'TV_22': float,
    'Est_Taxes_22': float,
    'MV_change': float,
    'MV_per_change': float,
    'Est_Taxes_change': float,
    'Est_Taxes_per_change': float,
}
joined = pd.read_csv('./processed/joined-on-geocode.csv', dtype=dtype_joined)
joined['State'] = 'Montana' # Makes statewide pivot table easier

dtype_components = {
    'TaxYear': str,
    'Co': str,
    'County': str,
    'LevyDistrictCode': str,
    'PropertyNumber': str,
    'TaxClass': str,
    'ClassCode': str,
    'ClassCodeDesc': str,
    'AbateInd': str,
    'SM': str,
    'PropertyTypeDesc': str,
    'PropertyCat_mtfp': str, # MTFP-assigned in earlier step
    'TIFName': str,
    'TIFCode': str,
    'AssessmentCode': str,
    'NameLast': str,
    'Address1': str,
    'Address2': str,
    'Address3': str,
    'City': str,
    'State': str,
    'ZIP': str,
    'Situs_Address': str,
    'Situs_City': str,
    'Situs_State': str,
    'Situs_ZipCode': str,
    'MV': float,
    'TV': float,
    'Mills': float,
    'Est_Taxes': float,
}
property_components_22 = pd.read_csv('./processed/2022-property-components.csv', dtype=dtype_components)
property_components_23 = pd.read_csv('./processed/2023-property-components.csv', dtype=dtype_components)

/var/folders/gk/_q4g8knj7354h1qcc0gsdcq00000gp/T/ipykernel_24741/3376993969.py:59: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  property_components_23 = pd.read_csv('./processed/2023-property-components.csv', dtype=dtype_components)


In [3]:
# Sample data row - joined properties
joined.head(1)

,County,PropertyNumber,PropertyTypeDesc_23,PropertyCat_mtfp_23,NameLast_23,TaxClass_23,ClassCodeDesc_23,MV_23,TV_23,Est_Taxes_23,PropertyTypeDesc_22,PropertyCat_mtfp_22,NameLast_22,TaxClass_22,ClassCodeDesc_22,MV_22,TV_22,Est_Taxes_22,MV_change,MV_per_change,Est_Taxes_change,Est_Taxes_per_change,State
0,BEAVERHEAD,18-0000008172-001,Personal Property Attached to Real Property,Other,HARRINGTON COMPANY,8,"6111 - Agricultural Implements & Machinery, 65...","52,801.00",792.00,387.29,Personal Property Attached to Real Property,Other,HARRINGTON COMPANY,8,"6111 - Agricultural Implements & Machinery, 65...","42,892.00",643.00,367.19,"9,909.00",0.23,20.10,0.05,Montana


In [4]:
# Sample data row - property components
property_components_23.head(1)

,TaxYear,Co,LevyDistrictCode,PropertyNumber,TaxClass,ClassCode,ClassCodeDesc,AbateInd,SM,PropertyTypeDesc,TIFName,TIFCODE,AssessmentCode,NameLast,Address1,Address2,Address3,City,State,ZIP,Situs_Address,Situs_City,Situs_State,Situs_ZipCode,MV,TV,Mills,County,state_mills,Mills_unadjusted,Est_Taxes,PropertyCat_mtfp
0,2023,1,01-0453,01-0889-12-1-02-01-0000,3,1601,1601 - Grazing Land,N,No,Agricultural and Timber Properties,NaN,NaN,0001677900,HOLT & BAKER RANCHES,70 FRANICH LN,NaN,NaN,WHITEHALL,MT,59759-8649,NaN,NaN,NaN,NaN,"6,081.00",131.00,538.93,SILVER BOW,77.89,521.82,70.60,Agricultural


## a) Tax changes for existing properties

Properties in the "joined" data table (gecodes present in both 2022 and 2023)

### i) All property types

In [5]:
# All properties statewide

joined_statewide = joined.pivot_table(
    values=['PropertyNumber','Est_Taxes_22','Est_Taxes_23','Est_Taxes_change','Est_Taxes_per_change'],
    index=['State'],
    aggfunc={
        'PropertyNumber': 'count',
        'Est_Taxes_per_change': [per25, per50, per75],
        'Est_Taxes_change': ['sum', per25, per50, per75],
        'Est_Taxes_23': ['sum'],
        'Est_Taxes_22': ['sum'],
    }
)
joined_statewide

Est_Taxes_22     Est_Taxes_23 Est_Taxes_change               \
                     sum              sum            per25 per50  per75   
State                                                                     
Montana 2,080,135,358.96 2,319,618,966.13            -0.14 26.38 380.06   

                       Est_Taxes_per_change             PropertyNumber  
                   sum                per25 per50 per75          count  
State                                                                   
Montana 239,483,607.17                -0.03  0.12  0.27         955896

Takeaways:
- About 956k properties in this joined data
- Est. tax collections increased on aggregated $239M between 2022 and 2023

In [6]:
# Broken down by MTFP-assigned simplified property category
# See 01-join-property-components.ipynb for how these simplified categories map to the ones used by the Dept. of Revenue

joined_by_prop_cat = joined.pivot_table(
    values=['PropertyNumber','Est_Taxes_change','Est_Taxes_22','Est_Taxes_per_change'],
    index=['PropertyCat_mtfp_23'],
    # index='PropertyTypeDesc_23', # more detailed categories used by DOR; too detailed for a general audience
    aggfunc={
        'PropertyNumber': 'count',
        'Est_Taxes_per_change': [per25, per50, per75],
        'Est_Taxes_change': ['sum', per25, per50, per75],
        # 'Est_Taxes_22': ['sum'],
    }
)
joined_by_prop_cat.sort_values(('PropertyNumber','count'), ascending=False)

Est_Taxes_change                               \
                               per25  per50  per75            sum   
PropertyCat_mtfp_23                                                 
Residential                    97.82 345.05 660.44 213,178,550.13   
Agricultural                  -17.75  -1.29   8.55  13,573,999.81   
Other                           0.00   0.00   5.09  11,783,173.04   
Commercial                      0.00 197.88 771.81  53,870,448.85   
Industrial                     -9.43   1.78  19.92 -52,922,564.65   

                    Est_Taxes_per_change             PropertyNumber  
                                   per25 per50 per75          count  
PropertyCat_mtfp_23                                                  
Residential                         0.11  0.21  0.35         410876  
Agricultural                       -0.09 -0.03  0.06         242535  
Other                              -0.10  0.07  0.30         234007  
Commercial                          0.01  0.12  0.29          58182  
Industrial                         -0.13  0.06  0.17          10296

Takeaways:
- Typical residential properties saw tax bills 21% higher in 2022 than 2023 (11%–35% at 25th–75th percentiles)
- Median residential tax bill rose $345
- Aggregate taxes paid by existing residential property owners grew by $213 million
- Commercial saw a comparatively modest increase, up on median 12%
- Ag decreased on median, but increased in aggregate
- Some but not all industrial properties are down. Median change is +6%, but aggregate tax bill for existing industrial properties is down by $53 million.

*Note: Residential here = "Residential" in MTFP's simplified property categories, which is analogous to "Residential" in DOR `PropertyTypeDesc_23` field. It excludes mobile homes and residences on non-residential (e.g. Ag) property. Industrial bundles several DOR types — see `01-join-property-components.ipynb`*

In [7]:
# County by county, all properties
all_props_by_county = joined.pivot_table(
    values=['PropertyNumber','MV_23','MV_change','MV_per_change','Est_Taxes_change','Est_Taxes_per_change'],
    index='County',
    aggfunc={
        'PropertyNumber': 'count',
        'MV_23': ['median'],
        'MV_change': ['median'],
        'Est_Taxes_change': ['sum', per25, per50, per75],
        'Est_Taxes_per_change': ['median'],
    },
)

# Not an especially interesting table
# all_props_by_county.sort_values(('PropertyNumber','count'), ascending=False).head(10)

### ii) Tax changes for residential properties

In [8]:
# Homes = residential propties
homes = joined[joined['PropertyCat_mtfp_23'] == 'Residential']

# Possible alternate approach — any property that has a class 4.1 structure component on it
# homes = joined[joined['TaxClass_23'].str.contains('4.1')]
# homes['PropertyCat_mtfp_23'].value_counts()

In [9]:
# All "home" properties statewide

homes_statewide = homes.pivot_table(
    values=['PropertyNumber','Est_Taxes_22','Est_Taxes_change','Est_Taxes_per_change'],
    index=['State'],
    aggfunc={
        'PropertyNumber': 'count',
        'Est_Taxes_per_change': [per25, per50, per75],
        'Est_Taxes_change': ['sum', per25, per50, per75],
        'Est_Taxes_22': ['sum']
    }
)
homes_statewide['Est_Taxes_per_change_avg'] = homes_statewide['Est_Taxes_change','sum'] / homes_statewide['Est_Taxes_22','sum']
homes_statewide

Est_Taxes_22 Est_Taxes_change                               \
                   sum            per25  per50  per75            sum   
State                                                                  
Montana 872,112,735.54            97.82 345.05 660.44 213,178,550.13   

        Est_Taxes_per_change             PropertyNumber  \
                       per25 per50 per75          count   
State                                                     
Montana                 0.11  0.21  0.35         410876   

        Est_Taxes_per_change_avg  
                                  
State                             
Montana                     0.24

In [10]:
homes_by_county = homes.pivot_table(
    values=[
        'PropertyNumber',
        'MV_23',
        'MV_change',
        'MV_per_change',
        'Est_Taxes_23',
        'Est_Taxes_change',
        'Est_Taxes_per_change',
        ],
    index='County',
    aggfunc={
        'PropertyNumber': 'count',
        # 'MV_23': [per50],
        # 'MV_change': ['sum', per25, per50, per75],
        'Est_Taxes_23': ['sum', per25, per50, per75],
        'Est_Taxes_change': ['sum', per25, per50, per75],
        'MV_per_change': [per25, per50, per75],
        'Est_Taxes_per_change': [per25, per50, per75],
    },
).sort_values(('Est_Taxes_change', 'sum'), ascending=False)

homes_by_county.sort_values(('PropertyNumber','count'), ascending=False)
homes_by_county.sort_values(('Est_Taxes_per_change','per50'), ascending=False)

Est_Taxes_23                                   \
                       per25    per50    per75            sum   
County                                                          
JUDITH BASIN           53.12    79.74   539.34     649,690.82   
MADISON               418.66 1,593.72 3,345.64  56,888,690.81   
BROADWATER            592.33 1,831.25 3,126.77   6,378,004.26   
GRANITE               469.35 1,030.92 2,071.37   4,386,935.25   
JEFFERSON             625.04 2,230.05 3,418.20  12,034,562.68   
SANDERS               385.75 1,054.79 2,019.84   7,832,610.43   
SWEET GRASS         1,082.78 1,664.68 2,401.65   2,253,356.47   
PONDERA               225.09 1,080.46 1,906.12   2,336,410.03   
POWELL                363.12 1,120.19 1,855.74   4,071,380.81   
MEAGHER               294.62   682.95 1,631.64   1,512,337.65   
STILLWATER            581.45 1,676.28 2,455.31   6,380,616.88   
GALLATIN            2,984.24 4,062.16 5,279.32 188,590,151.37   
MINERAL               452.18 1,074.88 2,341.86   3,507,045.74   
BLAINE                 77.03   700.75 1,891.01   1,882,803.61   
VALLEY                 96.29   407.99 1,607.51   4,513,378.96   
GLACIER               299.19   757.90 1,595.59   2,980,469.28   
LAKE                  957.15 2,158.50 3,525.22  35,746,112.24   
BIG HORN              196.50   958.09 1,763.80   2,661,907.62   
CARBON                533.67 1,502.23 2,468.59  11,062,285.36   
PARK                1,477.09 2,383.54 3,053.82  17,390,776.52   
SILVER BOW            753.22 1,621.49 2,445.83  25,513,812.20   
DAWSON                343.71 1,319.44 2,127.89   4,964,482.47   
LEWIS AND CLARK     1,975.67 3,004.97 3,816.62  73,493,804.54   
BEAVERHEAD            742.41 1,869.33 2,615.99   7,107,799.28   
WHEATLAND              96.16   318.80   762.55     544,581.44   
LIBERTY                38.25   554.78 1,228.59     510,353.11   
GOLDEN VALLEY          35.26   125.06   741.14     157,538.69   
DEER LODGE            640.66 1,146.42 1,943.23   6,757,351.91   
CASCADE             1,226.21 2,089.54 2,761.48  59,546,856.58   
CHOUTEAU              112.26   591.54 1,510.76   1,974,409.73   
FERGUS                230.42 1,086.48 2,164.36   7,095,048.28   
POWDER RIVER          136.35   297.54   796.32     229,282.90   
LINCOLN               381.84 1,007.09 1,792.34  14,701,304.67   
TOOLE                 111.23   579.45 1,432.87   1,948,279.42   
YELLOWSTONE         1,896.19 2,637.44 3,365.81 144,235,486.83   
RAVALLI             1,184.58 2,136.64 2,865.35  37,206,769.13   
MISSOULA            3,066.27 4,099.66 5,253.94 152,827,236.30   
ROSEBUD                78.28   369.83   980.21   1,501,269.93   
PETROLEUM              50.60   113.79   347.39     109,330.55   
TETON                 316.13 1,011.44 1,645.81   2,411,992.25   
FLATHEAD            1,604.21 2,614.58 3,573.21 142,176,719.97   
CUSTER                402.38 1,387.77 2,220.96   6,577,149.11   
WIBAUX                 63.93   189.88   529.25     143,958.18   
HILL                  516.65 1,408.35 2,030.54   6,970,866.65   
ROOSEVELT              93.76   491.76 1,124.45   1,667,531.52   
GARFIELD              106.01   586.75 1,288.18     409,471.25   
CARTER                 97.33   166.93   452.12     147,080.16   
PHILLIPS              167.15   532.13 1,400.39   1,646,288.82   
MCCONE                 84.00   506.76 1,252.68     588,219.00   
TREASURE               97.52   380.93 1,048.93     171,526.40   
RICHLAND              289.29 1,089.27 1,785.88   4,594,563.55   
PRAIRIE                26.54   116.44   584.42     288,446.49   
SHERIDAN               88.59   484.25 1,298.32   1,680,858.95   
DANIELS                45.81   365.70 1,139.62     769,513.91   
MUSSELSHELL            60.46   208.15   669.23     873,334.04   
FALLON                161.74   410.31   925.40     689,240.67   

                Est_Taxes_change                                \
                           per25  per50    per75           sum   
County                                          

In [11]:
# flattening column hierarchy to allow for simpler export later
joined_statewide.columns = joined_statewide.columns.map('_'.join) 
joined_by_prop_cat.columns = joined_by_prop_cat.columns.map('_'.join) 
all_props_by_county.columns = all_props_by_county.columns.map('_'.join)
homes_by_county.columns = homes_by_county.columns.map('_'.join)

## Tax changes at a tax base level

This analysis uses property components data rather than joined property data in an effort to reflect tax bases in their entirity.

## 

In [12]:
# Compare shifts in tax base composition county-by-county

mv_by_county_23 = property_components_23.pivot_table(
    values=['MV'],
    index=['County'],
    aggfunc={'MV': ['sum'],},
)['MV','sum'].to_dict()
mv_by_county_22 = property_components_22.pivot_table(
    values=['MV'],
    index=['County'],
    aggfunc={'MV': ['sum'],},
)['MV','sum'].to_dict()

taxes_by_county_23 = property_components_23.pivot_table(
    values=['Est_Taxes'],
    index=['County'],
    aggfunc={'Est_Taxes': ['sum'],},
)['Est_Taxes','sum'].to_dict()
taxes_by_county_22 = property_components_22.pivot_table(
    values=['Est_Taxes'],
    index=['County'],
    aggfunc={'Est_Taxes': ['sum'],},
)['Est_Taxes','sum'].to_dict()

aggregate_tax_base_by_county_23 = property_components_23.pivot_table(
    values=['MV','Est_Taxes'],
    index=['County', 'PropertyCat_mtfp'],
    aggfunc={
        'MV': ['sum'],
        'Est_Taxes': ['sum'],
    },
)
aggregate_tax_base_by_county_22 = property_components_22.pivot_table(
    values=['MV','Est_Taxes'],
    index=['County', 'PropertyCat_mtfp'],
    aggfunc={
        'MV': ['sum'],
        'Est_Taxes': ['sum'],
    },
)

aggregate_tax_base_by_county_23['MV','fraction'] = aggregate_tax_base_by_county_23.apply( lambda row: row['MV','sum'] / mv_by_county_23[row.name[0]], axis=1)
aggregate_tax_base_by_county_22['MV','fraction'] = aggregate_tax_base_by_county_22.apply( lambda row: row['MV','sum'] / mv_by_county_22[row.name[0]], axis=1)
aggregate_tax_base_by_county_23['Est_Taxes','fraction'] = aggregate_tax_base_by_county_23.apply( lambda row: row['Est_Taxes','sum'] / taxes_by_county_23[row.name[0]], axis=1)
aggregate_tax_base_by_county_22['Est_Taxes','fraction'] = aggregate_tax_base_by_county_22.apply( lambda row: row['Est_Taxes','sum'] / taxes_by_county_22[row.name[0]], axis=1)

aggregate_tax_base_by_county = aggregate_tax_base_by_county_23.merge(aggregate_tax_base_by_county_22, 
    left_index=True, 
    right_index=True,
    suffixes=['_23','_22']
)

aggregate_tax_base_by_county['MV_fraction_delta'] = aggregate_tax_base_by_county['MV_23','fraction'] - aggregate_tax_base_by_county['MV_22','fraction']
aggregate_tax_base_by_county['Est_Taxes_fraction_delta'] = aggregate_tax_base_by_county['Est_Taxes_23','fraction'] - aggregate_tax_base_by_county['Est_Taxes_22','fraction']

# Too complex to visualize as a table, so preparing for export for visualization elsewhere instead
aggregate_tax_base_by_county.columns = aggregate_tax_base_by_county.columns.map('_'.join)

In [13]:
# Total estimated taxes by county

aggregate_collections_by_county_23 = property_components_23.pivot_table(
    values=['MV','TV','Est_Taxes'],
    index=['County'],
    aggfunc={
        'MV': ['sum'],
        'TV': ['sum'],
        'Est_Taxes': ['sum'],
    },
)
aggregate_collections_by_county_22 = property_components_22.pivot_table(
    values=['MV','TV','Est_Taxes'],
    index=['County'],
    aggfunc={
        'MV': ['sum'],
        'TV': ['sum'],
        'Est_Taxes': ['sum'],
    },
)
aggregate_collections_by_county = aggregate_collections_by_county_23.merge(aggregate_collections_by_county_22, 
    left_index=True, 
    right_index=True,
    suffixes=['_23','_22']
)
aggregate_collections_by_county

aggregate_collections_by_county['Est_MV_per_change'] = aggregate_collections_by_county['MV_23','sum'] / aggregate_collections_by_county['MV_22','sum'] - 1
aggregate_collections_by_county['Est_TV_per_change'] = aggregate_collections_by_county['TV_23','sum'] / aggregate_collections_by_county['TV_22','sum'] - 1
aggregate_collections_by_county['Est_Taxes_per_change'] = aggregate_collections_by_county['Est_Taxes_23','sum'] / aggregate_collections_by_county['Est_Taxes_22','sum'] - 1
aggregate_collections_by_county['Est_Taxes_change'] = aggregate_collections_by_county['Est_Taxes_23','sum'] - aggregate_collections_by_county['Est_Taxes_22','sum']
aggregate_collections_by_county.columns = aggregate_collections_by_county.columns.map('_'.join)


aggregate_collections_by_county[
    ['Est_Taxes_23_sum','Est_Taxes_22_sum','Est_Taxes_change_', 'Est_Taxes_per_change_','Est_MV_per_change_', 'Est_TV_per_change_']
].sort_values('Est_Taxes_per_change_', ascending=False)


,Est_Taxes_23_sum,Est_Taxes_22_sum,Est_Taxes_change_,Est_Taxes_per_change_,Est_MV_per_change_,Est_TV_per_change_
County,,,,,,
MADISON,"72,907,478.51","52,163,362.14","20,744,116.38",0.40,0.67,0.64
GALLATIN,"324,815,540.40","255,641,640.14","69,173,900.26",0.27,0.66,0.60
LAKE,"50,733,341.06","41,005,030.33","9,728,310.73",0.24,0.43,0.43
BIG HORN,"20,818,075.05","17,477,735.74","3,340,339.31",0.19,0.06,0.08
BROADWATER,"13,697,113.51","11,654,816.79","2,042,296.72",0.18,0.51,0.35
JUDITH BASIN,"9,828,728.49","8,457,451.40","1,371,277.09",0.16,0.15,0.08
JEFFERSON,"22,237,787.37","19,137,144.95","3,100,642.41",0.16,0.43,0.32
GARFIELD,"4,849,860.37","4,178,546.93","671,313.44",0.16,0.40,0.36
BEAVERHEAD,"17,760,068.11","15,372,684.16","2,387,383.96",0.16,0.38,0.35


Takeaways:
- Some counties actually collected less in taxes this year than last (aggregate figures for all taxes collected in the county)
- Gallatin County now has a bigger aggregate tax bill than Yellowstone Co
- Collections generally rose more slower than both MV and TV

In [14]:
# Aggregate collections by tax class

aggregate_collections_by_class_23 = property_components_23.pivot_table(
    values=['MV','TV','Est_Taxes'],
    index=['TaxClass'],
    aggfunc={
        'MV': ['sum'],
        'TV': ['sum'],
        'Est_Taxes': ['sum'],
    },
)
aggregate_collections_by_class_22 = property_components_22.pivot_table(
    values=['MV','TV','Est_Taxes'],
    index=['TaxClass'],
    aggfunc={
        'MV': ['sum'],
        'TV': ['sum'],
        'Est_Taxes': ['sum'],
    },
)
aggregate_collections_by_class = aggregate_collections_by_class_23.merge(aggregate_collections_by_class_22, 
    left_index=True, 
    right_index=True,
    suffixes=['_23','_22']
)

aggregate_collections_by_class['Est_MV_per_change'] = aggregate_collections_by_class['MV_23','sum'] / aggregate_collections_by_class['MV_22','sum'] - 1
aggregate_collections_by_class['Est_TV_per_change'] = aggregate_collections_by_class['TV_23','sum'] / aggregate_collections_by_class['TV_22','sum'] - 1
aggregate_collections_by_class['Est_Taxes_per_change'] = aggregate_collections_by_class['Est_Taxes_23','sum'] / aggregate_collections_by_class['Est_Taxes_22','sum'] - 1
aggregate_collections_by_class['Est_Taxes_change'] = aggregate_collections_by_class['Est_Taxes_23','sum'] - aggregate_collections_by_class['Est_Taxes_22','sum']
aggregate_collections_by_class.columns = aggregate_collections_by_class.columns.map('_'.join)

TAX_CLASSES = {
    '1': 'Mine proceeds, net',
    '2': 'Mine proceeds, gross',
    '3': 'Agricultural land',
    '4.1': 'Residential improvements',
    '4.2': 'Residential land',
    '4.8': 'Commercial improvements',
    '4.9': 'Commercial land',
    '5': 'Misc industrial', # Complex code
    '7': 'Noncentrally assessed utiltiies',
    '8': 'Business equipment',
    '9': 'Pipelines and non-gen electric',
    '10': 'Forest land',
    '12': 'Airlines and railroads',
    '13': 'Telecom and electric generation',
    '14': 'Renewable energy',
    '15': 'CO2 and liquid pipeline',
    '16': 'High voltage DC converter',
    '17': 'Qualified data centers',
    '18': 'Green Hydrogen facilities',
}

aggregate_collections_by_class['ClassDesc'] = aggregate_collections_by_class.apply(lambda row: TAX_CLASSES[row.name], axis=1)

aggregate_collections_by_class[
    ['ClassDesc', 'Est_Taxes_23_sum','Est_Taxes_22_sum','Est_Taxes_change_', 'Est_Taxes_per_change_','Est_MV_per_change_', 'Est_TV_per_change_']
].sort_values('Est_Taxes_23_sum', ascending=False)

,ClassDesc,Est_Taxes_23_sum,Est_Taxes_22_sum,Est_Taxes_change_,Est_Taxes_per_change_,Est_MV_per_change_,Est_TV_per_change_
TaxClass,,,,,,,
4.1,Residential improvements,"997,580,044.89","801,474,997.07","196,105,047.82",0.24,0.45,0.48
4.2,Residential land,"354,497,539.79","279,833,431.51","74,664,108.28",0.27,0.55,0.57
9,Pipelines and non-gen electric,"267,073,607.85","308,031,200.66","-40,957,592.81",-0.13,-0.05,-0.05
4.8,Commercial improvements,"241,803,945.22","218,717,111.07","23,086,834.15",0.11,0.32,0.29
4.9,Commercial land,"114,460,328.34","98,629,426.11","15,830,902.23",0.16,0.38,0.40
8,Business equipment,"107,036,408.82","85,969,467.24","21,066,941.57",0.25,0.35,0.36
3,Agricultural land,"79,603,295.69","83,292,794.11","-3,689,498.42",-0.04,-0.02,0.01
13,Telecom and electric generation,"65,266,134.39","79,849,209.66","-14,583,075.27",-0.18,-0.13,-0.14
12,Airlines and railroads,"49,206,977.69","56,328,880.74","-7,121,903.06",-0.13,-0.07,-0.07


Takeaways:
- Residential class 4 collections are up a bunch
- Commercial class 4 collections are up some
- Most industrial classification collections are down

*This is in line with what Bob Story presented at the last Revenue Interim Committee meeting.*


### Industrial taxpayers

In [20]:
# Major changes in industrial taxpayers

industrial = joined[joined['PropertyCat_mtfp_23'] == 'Industrial'].copy()
industrial_major_owners = industrial.pivot_table(
    values=['PropertyNumber', 'Est_Taxes_22','Est_Taxes_23', 'MV_23', 'MV_change', 'Est_Taxes_change'],
    index='NameLast_23',
    aggfunc={
        'PropertyNumber': 'count',
        'MV_23': ['sum'],
        'MV_change': ['sum'],
        'Est_Taxes_22': ['sum'],
        'Est_Taxes_23': ['sum'],
        'Est_Taxes_change': ['sum'],
    }
)
industrial_major_owners['est_taxes_percent_change'] = industrial_major_owners['Est_Taxes_change', 'sum'] / industrial_major_owners['Est_Taxes_22', 'sum']
industrial_major_owners.sort_values(('Est_Taxes_change','sum'), ascending=True).head(20) # Show 10 biggest absolute dollar declines 


,Est_Taxes_22,Est_Taxes_23,Est_Taxes_change,MV_23,MV_change,PropertyNumber,est_taxes_percent_change
,sum,sum,sum,sum,sum,count,
NameLast_23,,,,,,,
NORTHWESTERN ENERGY-T & D,"149,767,828.58","118,070,926.65","-31,696,901.94","1,825,308,612.00","-215,952,158.00",41,-0.21
NORTHWESTERN ENERGY - ELECTRIC GENERATION,"24,843,468.44","21,145,826.39","-3,697,642.05","744,518,162.00","-63,019,181.00",12,-0.15
MONTANA RAIL LINK,"10,345,676.83","7,450,622.03","-2,895,054.79","492,455,377.00","-121,089,932.00",33,-0.28
VERIZON INC,"10,369,600.13","7,775,722.19","-2,593,877.95","226,469,975.00","-39,259,064.00",53,-0.25
LUMEN TECHNOLOGIES INC DB,"6,645,291.75","4,607,086.81","-2,038,204.95","134,046,474.00","-35,654,636.00",39,-0.31
CHARTER COMMUNICATIONS INC,"8,491,941.55","6,736,524.35","-1,755,417.20","193,188,570.00","-13,761,716.00",31,-0.21
AVISTA CORPORATION - ELECTRIC GENERATION,"6,898,604.67","5,195,851.10","-1,702,753.56","219,941,751.00","-80,748,793.00",2,-0.25
BNSF RAILWAY CO,"37,246,818.46","35,605,507.81","-1,641,310.66","2,120,478,399.00","-8,781,532.00",46,-0.04


In [16]:
# Northwestern Energy properties
nwe = joined[
    (joined['NameLast_23'].str.contains('NORTHWESTERN ENERGY')
      | (joined['NameLast_23'] == 'NORTHWESTERN CORPORATION'))
]
print(len(nwe), 'properties')
nwe[['Est_Taxes_22','Est_Taxes_23', 'MV_change', 'Est_Taxes_change']].sum()

1487 properties


Est_Taxes_22        176,527,868.27
Est_Taxes_23        140,836,853.94
MV_change          -289,128,467.00
Est_Taxes_change    -35,691,014.34
dtype: float64

In [17]:
# Charter Communications properties
cc = joined[joined['NameLast_23'].str.contains('CHARTER COMMUNICATIONS')]
print(len(cc), 'properties')
cc[['Est_Taxes_22','Est_Taxes_23', 'MV_change', 'Est_Taxes_change']].sum()

31 properties


Est_Taxes_22         8,491,941.55
Est_Taxes_23         6,736,524.35
MV_change          -13,761,716.00
Est_Taxes_change    -1,755,417.20
dtype: float64

In [18]:
# Montana Rail Link properties
mrl = joined[joined['NameLast_23'].str.contains('MONTANA RAIL LINK')]
print(len(mrl), 'properties')
mrl[['Est_Taxes_22','Est_Taxes_23', 'MV_change', 'Est_Taxes_change']].sum()

456 properties


Est_Taxes_22        11,461,678.82
Est_Taxes_23         8,934,434.00
MV_change          -76,440,135.00
Est_Taxes_change    -2,527,244.82
dtype: float64

### Exports

In [19]:
# Aggregate data tables and export as JSON for use in data visualization products

hbc = homes_by_county.to_dict(orient='index')
atbc = aggregate_tax_base_by_county.reset_index().to_dict(orient='records')
acbc = aggregate_collections_by_county.to_dict(orient='index')

counties = [{
    'county': county,
    'residential': hbc[county],
    'tot_by_prop_cat': [d for d in atbc if d['County'] == county],
    'tot_collections': acbc[county],

} for county in list(homes_by_county.index)]


output = {
    'statewide': {
        'joined_aggregate': joined_statewide.to_dict(orient='records')[0],
        'joined_by_prop_cat': joined_by_prop_cat.reset_index().to_dict(orient='records'),
    },
    'counties': counties,
}

with open('outputs/summary.json', 'w') as fp:
    json.dump(output, fp)
    print('Done')


Done
